In [46]:
print("hii")

hii


In [47]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()



In [48]:
print(len(text))

232309


In [49]:
print(text[:200])

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

  ILLUSTRATED BY JOHN R. NEILL

  BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW


In [50]:
from pprint import pprint


chars = sorted(set(text))
pprint(chars)

['\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\ufeff']


In [51]:
print(len(chars))

81


In [52]:

string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [53]:
encode('hello')

[61, 58, 65, 65, 68]

In [54]:
decode(encode('hello'))

'hello'

Character Level Tokenizer - 
converts each character to integer equivalents.

in this case, we have very small vocabulary and very large amount of tokens to convert (large number of predictions would be required to generate the text)

Word Level Tokenizer - 
we would map each word in a set of vocabulary to integers.

in this case the vocabulary would be very large, but less number of tokens to encode/decode

Sub-Word Tokenizer - 
in-between character and word level

In [55]:
import torch

ModuleNotFoundError: No module named 'torch'

In [ ]:
x = torch.rand(5, 3)
print(x)